# Procesamiento de Lenguaje Natural I

**Autor:** Gonzalo G. Fernandez

Clase 3: Modelos secuenciales - RNNs y LSTM

## Consigna desafío 3
- Seleccionar un corpus de texto sobre el cual entrenar el modelo de lenguaje.
- Realizar el pre-procesamiento adecuado para tokenizar el corpus, estructurar el dataset y separar entre datos de entrenamiento y validación.
- Proponer arquitecturas de redes neuronales basadas en unidades recurrentes para implementar un modelo de lenguaje.
- Con el o los modelos que consideren adecuados, generar nuevas secuencias a partir de secuencias de contexto con las estrategias de greedy search y beam search determístico y estocástico. En este último caso observar el efecto de la temperatura en la generación de secuencias.

**Sugerencias:**

- Durante el entrenamiento, guiarse por el descenso de la perplejidad en los datos de validación para finalizar el entrenamiento. Para ello se provee un callback.
- Explorar utilizar SimpleRNN (celda de Elman), LSTM y GRU.
- rmsprop es el optimizador recomendado para la buena convergencia. No obstante se pueden explorar otros.

In [32]:
from utils import parse_srt_to_dialogue

import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import (
    SimpleRNN,
    Dense,
    Dense,
    LSTM,
    GRU,
    Dropout,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K

import keras_tuner as kt

## Resolución

Se utilizarán los diálogos de la película argentina "Nueve Reinas" del año 2000 como corpus.

In [2]:
dialogues = parse_srt_to_dialogue("data/nueve_reinas-subtitles.srt")
print(f"Total dialogues extracted: {len(dialogues)}")
for d in dialogues[:5]:
    print(d)
corpus = " ".join(dialogues).lower()

Total dialogues extracted: 1371
¿Qué estás leyendo?
Nada... disculpáme. No, está todo bien.
¿Nada más que esto? Sí.
Esta máquina me vuelve loca. Después lo registro.
1.25 más 3.75 son... 5


### Preprocesamiento

Tokenización de caracteres.

In [3]:
chars = sorted(list(set(corpus)))
char2idx = {char: idx for idx, char in enumerate(chars)}
idx2char = {idx: char for char, idx in char2idx.items()}
vocab_size = len(chars)

tokenized_corpus = [char2idx[c] for c in corpus]

Creación de secuencias de entrada y labels.

In [4]:
seq_length = 100
step = 1

sequences = []
next_chars = []

for i in range(0, len(tokenized_corpus) - seq_length, step):
    sequences.append(tokenized_corpus[i : i + seq_length])
    next_chars.append(tokenized_corpus[i + seq_length])

X = np.array(sequences)
y = np.array(next_chars)

- División del dataset en entrenamiento y validación.
- One-Hot encoding de las secuencias de entrada y los labels.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

X_train_encoded = to_categorical(X_train, num_classes=vocab_size)
y_train_encoded = to_categorical(y_train, num_classes=vocab_size)

X_test_encoded = to_categorical(X_test, num_classes=vocab_size)
y_test_encoded = to_categorical(y_test, num_classes=vocab_size)

Se utiliza como métrica de desempeño la perplejidad.

In [6]:
def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    return K.exp(K.mean(cross_entropy))

### Creación y entrenamiento de modelo SimpleRNN

In [26]:
model_simple_rnn = Sequential(
    [
        SimpleRNN(128, input_shape=(seq_length, vocab_size)),
        Dense(vocab_size, activation="softmax"),
    ]
)

model_simple_rnn.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy", perplexity],
)

model_simple_rnn.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    batch_size=64,
    epochs=15,
)

/home/ggf/workspace/ceia/ceia-nlp1/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.2373 - loss: 2.8112 - perplexity: 17.7516 - val_accuracy: 0.3296 - val_loss: 2.2973 - val_perplexity: 10.0630
Epoch 2/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 15s 23ms/step - accuracy: 0.3468 - loss: 2.2141 - perplexity: 9.2785 - val_accuracy: 0.3625 - val_loss: 2.1467 - val_perplexity: 8.6727
Epoch 3/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.3680 - loss: 2.0921 - perplexity: 8.2185 - val_accuracy: 0.3700 - val_loss: 2.0949 - val_perplexity: 8.2272
Epoch 4/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 14s 21ms/step - accuracy: 0.3855 - loss: 2.0192 - perplexity: 7.6324 - val_accuracy: 0.3831 - val_loss: 2.0424 - val_perplexity: 7.8207
Epoch 5/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.3992 - loss: 1.9778 - perplexity: 7.3403 - val_accuracy: 0.3886 - val_loss: 2.0199 - val_perplexity: 7.6431
Epoch 6/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - accuracy: 0.4053 - loss: 1.9440 - perplexity: 7.0860 - val_acc

- Dado que el vocabulario es de 54, una perplejidad de ~6 indica relativamente buen desempeño.
- El entrenamiento del modelo es relativamente rápido.

### Creación y entrenamiento de modelo LSTM

In [17]:
model_lstm = Sequential(
    [
        LSTM(128, input_shape=(seq_length, vocab_size)),
        Dense(vocab_size, activation="softmax"),
    ]
)

model_lstm.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy", perplexity],
)

model_lstm.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    batch_size=64,
    epochs=15,
)

/home/ggf/workspace/ceia/ceia-nlp1/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 60s 91ms/step - accuracy: 0.2051 - loss: 2.9962 - perplexity: 21.4083 - val_accuracy: 0.3153 - val_loss: 2.3665 - val_perplexity: 10.7855
Epoch 2/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 55s 85ms/step - accuracy: 0.3158 - loss: 2.3149 - perplexity: 10.2629 - val_accuracy: 0.3452 - val_loss: 2.1764 - val_perplexity: 8.9133
Epoch 3/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 71s 110ms/step - accuracy: 0.3512 - loss: 2.1461 - perplexity: 8.6584 - val_accuracy: 0.3649 - val_loss: 2.1033 - val_perplexity: 8.2898
Epoch 4/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 71s 110ms/step - accuracy: 0.3715 - loss: 2.0685 - perplexity: 8.0215 - val_accuracy: 0.3752 - val_loss: 2.0417 - val_perplexity: 7.8011
Epoch 5/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 67s 104ms/step - accuracy: 0.3843 - loss: 2.0133 - perplexity: 7.5834 - val_accuracy: 0.3894 - val_loss: 1.9895 - val_perplexity: 7.4037
Epoch 6/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 67s 103ms/step - accuracy: 0.3990 - loss: 1.9580 - perplexity: 7.1937 - va

- Puede observarse como el tiempo de entrenamiento es mucho mayor que con SimpleRNN.

### Creación y entrenamiento de modelo GRU (Gated Recurrent Unit)
En búsqueda de un mejor desempeño que con LSTM.

In [19]:
model_gru = Sequential(
    [
        GRU(128, input_shape=(seq_length, vocab_size)),
        Dense(vocab_size, activation="softmax"),
    ]
)

model_gru.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.001),
    metrics=["accuracy", perplexity],
)

model_gru.fit(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    batch_size=64,
    epochs=15,
)

/home/ggf/workspace/ceia/ceia-nlp1/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 72s 110ms/step - accuracy: 0.2329 - loss: 2.8720 - perplexity: 19.4547 - val_accuracy: 0.3395 - val_loss: 2.2389 - val_perplexity: 9.4813
Epoch 2/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 69s 106ms/step - accuracy: 0.3421 - loss: 2.1830 - perplexity: 9.0149 - val_accuracy: 0.3561 - val_loss: 2.1050 - val_perplexity: 8.3222
Epoch 3/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 71s 109ms/step - accuracy: 0.3705 - loss: 2.0717 - perplexity: 8.0656 - val_accuracy: 0.3858 - val_loss: 2.0120 - val_perplexity: 7.5826
Epoch 4/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 69s 106ms/step - accuracy: 0.3923 - loss: 1.9690 - perplexity: 7.2620 - val_accuracy: 0.4072 - val_loss: 1.9449 - val_perplexity: 7.0927
Epoch 5/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 70s 108ms/step - accuracy: 0.4183 - loss: 1.8931 - perplexity: 6.7330 - val_accuracy: 0.4216 - val_loss: 1.8960 - val_perplexity: 6.7504
Epoch 6/15
648/648 ━━━━━━━━━━━━━━━━━━━━ 73s 113ms/step - accuracy: 0.4331 - loss: 1.8408 - perplexity: 6.3994 - va

### Comparación de perplejidad de modelos

In [20]:
test_loss, test_acc, test_perplexity = model_simple_rnn.evaluate(X_test_encoded, y_test_encoded, verbose=0)
print(f"SimpleRNN Test Perplexity: {test_perplexity:.4f} (vocab_size={vocab_size})")

test_loss, test_acc, test_perplexity = model_lstm.evaluate(X_test_encoded, y_test_encoded, verbose=0)
print(f"LSTM Test Perplexity: {test_perplexity:.4f} (vocab_size={vocab_size})")

test_loss, test_acc, test_perplexity = model_gru.evaluate(X_test_encoded, y_test_encoded, verbose=0)
print(f"GRU Test Perplexity: {test_perplexity:.4f} (vocab_size={vocab_size})")

SimpleRNN Test Perplexity: 6.5584 (vocab_size=54)
LSTM Test Perplexity: 5.8016 (vocab_size=54)
GRU Test Perplexity: 5.5559 (vocab_size=54)


Ordenando los modelos de forma decreciente utilizando como métrica la perplejidad en el subset de test se obtiene la siguiente lista:

1. GRU
2. LSTM
3. SimpleRNN

Es importante destacar que solo observando la perplejidad el desempeño es muy similar y por lo tanto resalta SimpleRNN como un modelo facil de entrenar que ofrece buenos resultados.

### Búsqueda de hiper parámetros para los modelos en estudio

Implementación de función constructora del modelo

In [35]:
def model_builder(hp):
    model = Sequential()
    rnn_type = hp.Choice("rnn_type", ["SimpleRNN", "LSTM", "GRU"])
    units = hp.Int("units", min_value=64, max_value=256, step=64)
    dropout = hp.Float("dropout", 0.0, 0.3, step=0.1)
    learning_rate = hp.Choice("lr", [1e-2, 1e-3, 1e-4])

    RNN = getattr(tf.keras.layers, rnn_type)
    model.add(RNN(units, input_shape=(seq_length, vocab_size)))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(vocab_size, activation="softmax"))

    model.compile(
        optimizer=RMSprop(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=[perplexity],
    )
    return model

In [ ]:
tuner = kt.Hyperband(
    model_builder,
    objective=kt.Objective("val_perplexity", direction="min"),
    max_epochs=10,
    directory="output",
    project_name="9reinas_rnn_tuning",
)

tuner.search(
    X_train_encoded,
    y_train_encoded,
    validation_data=(X_test_encoded, y_test_encoded),
    epochs=10,
    batch_size=64,
)

Trial 3 Complete [00h 01m 11s]
val_perplexity: 19.270008087158203

Best val_perplexity So Far: 18.206436157226562
Total elapsed time: 00h 02m 28s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
GRU               |SimpleRNN         |rnn_type
64                |64                |units
0                 |0                 |dropout
0.001             |0.0001            |lr
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
353/648 ━━━━━━━━━━━━━━━━━━━━ 15s 53ms/step - loss: 3.1601 - perplexity: 25.6881

### Generación de texto

Implementación de greedy search para generación de texto.

In [21]:
def generate_text_greedy(model, seed, length=200):
    seed = seed.lower()
    generated = seed

    for _ in range(length):
        input_seq = [char2idx.get(c, 0) for c in seed[-seq_length:]]
        input_seq = to_categorical([input_seq], num_classes=vocab_size)

        preds = model.predict(input_seq, verbose=0)[0]

        next_idx = np.argmax(preds) # greedy
        next_char = idx2char[next_idx]

        generated += next_char
        seed += next_char

    return generated

Implementación de beam search para generación de texto.

In [23]:
def generate_text_beam_search(model, seed, length=200, beam_width=3):
    seed = seed.lower()
    sequences = [(seed, 0.0)]

    for _ in range(length):
        all_candidates = []

        for seq, score in sequences:
            input_seq = [char2idx.get(c, 0) for c in seq[-seq_length:]]
            input_seq = to_categorical([input_seq], num_classes=vocab_size)
            preds = model.predict(input_seq, verbose=0)[0]

            top_indices = np.argsort(preds)[-beam_width:] # beam search

            for idx in top_indices:
                next_char = idx2char[idx]
                prob = preds[idx]
                candidate = (seq + next_char, score + np.log(prob + 1e-8))  # log-sum
                all_candidates.append(candidate)

        sequences = sorted(all_candidates, key=lambda tup: tup[1], reverse=True)[:beam_width]

    return sequences[0][0]

Comparación de greedy search para generación de texto entre modelos:

In [27]:
print("SimpleRNN:", generate_text_greedy(model_simple_rnn, "la verdad es que ", length=50))
print("LSTM:", generate_text_beam_search(model_lstm, "la verdad es que ", length=50))
print("GRU:", generate_text_beam_search(model_gru, "la verdad es que ", length=50))

SimpleRNN: la verdad es que te cama te para me raguna. ¿qué pasa? esta para qu
LSTM: la verdad es que estás las pero no se lo que te vay a de puedo que 
GRU: la verdad es que no te lo que yo te lo que yo te lo que yo te lo qu


Comparación de beam search para generación de texto entre modelos:

In [28]:
print("SimpleRNN:", generate_text_beam_search(model_simple_rnn, "la verdad es que ", length=50))
print("LSTM:", generate_text_beam_search(model_lstm, "la verdad es que ", length=50))
print("GRU:", generate_text_beam_search(model_gru, "la verdad es que ", length=50))

SimpleRNN: la verdad es que tengo que hacer una pertendo un para te hacer. ¿qu
LSTM: la verdad es que estás las pero no se lo que te vay a de puedo que 
GRU: la verdad es que no te lo que yo te lo que yo te lo que yo te lo qu


Puede observarse cómo las palabras se generan en su mayoría correctamente con los tres modelos pero en todos los casos la oración generada carece de contenido semántico. Además, hay una tendencia a generar palabras de gran frecuancia (comunes) en el corpus.